# Dataframes parallel benchmarking using Jupyter
### Kristjan Lõhmus

In [1]:
import time
results = []

## Dask

In [2]:
import dask_queries

### Scale 1

In [3]:
#factor 1
factor = 1
data_path=f'./data/scale_factor_{factor}'
dask_results_dict = {'library': 'dask', 'scale': factor}

In [4]:
# query 1
start_time = time.time()
dask_queries.query_01(data_path, factor)
end_time = time.time()
dask_results_dict['query_01'] = end_time - start_time

In [5]:
# query 2
start_time = time.time()
dask_queries.query_02(data_path, factor)
end_time = time.time()
dask_results_dict['query_02'] = end_time - start_time

In [6]:
# query 3
start_time = time.time()
dask_queries.query_03(data_path, factor)
end_time = time.time()
dask_results_dict['query_03'] = end_time - start_time

In [7]:
# query 4
start_time = time.time()
dask_queries.query_04(data_path, factor)
end_time = time.time()
dask_results_dict['query_04'] = end_time - start_time

In [8]:
# query 5
start_time = time.time()
dask_queries.query_05(data_path, factor)
end_time = time.time()
dask_results_dict['query_05'] = end_time - start_time

In [9]:
# query 6
start_time = time.time()
dask_queries.query_06(data_path, factor)
end_time = time.time()
dask_results_dict['query_06'] = end_time - start_time

In [10]:
# query 7
start_time = time.time()
dask_queries.query_07(data_path, factor)
end_time = time.time()
dask_results_dict['query_07'] = end_time - start_time

In [11]:
# query 8
start_time = time.time()
dask_queries.query_08(data_path, factor)
end_time = time.time()
dask_results_dict['query_08'] = end_time - start_time

In [12]:
# query 9
start_time = time.time()
dask_queries.query_09(data_path, factor)
end_time = time.time()
dask_results_dict['query_09'] = end_time - start_time

In [13]:
# query 10
start_time = time.time()
dask_queries.query_10(data_path, factor)
end_time = time.time()
dask_results_dict['query_10'] = end_time - start_time

In [14]:
# query 11
start_time = time.time()
dask_queries.query_11(data_path, factor)
end_time = time.time()
dask_results_dict['query_11'] = end_time - start_time

In [15]:
# query 12
start_time = time.time()
dask_queries.query_12(data_path, factor)
end_time = time.time()
dask_results_dict['query_12'] = end_time - start_time

In [16]:
# query 13
start_time = time.time()
dask_queries.query_13(data_path, factor)
end_time = time.time()
dask_results_dict['query_13'] = end_time - start_time

In [17]:
# query 14
start_time = time.time()
dask_queries.query_14(data_path, factor)
end_time = time.time()
dask_results_dict['query_14'] = end_time - start_time

In [18]:
# query 15
start_time = time.time()
dask_queries.query_15(data_path, factor)
end_time = time.time()
dask_results_dict['query_15'] = end_time - start_time

In [19]:
# query 16
start_time = time.time()
dask_queries.query_16(data_path, factor)
end_time = time.time()
dask_results_dict['query_16'] = end_time - start_time

In [20]:
# query 17
start_time = time.time()
dask_queries.query_17(data_path, factor)
end_time = time.time()
dask_results_dict['query_17'] = end_time - start_time

In [21]:
# query 18
start_time = time.time()
dask_queries.query_18(data_path, factor)
end_time = time.time()
dask_results_dict['query_18'] = end_time - start_time

In [22]:
# query 19
start_time = time.time()
dask_queries.query_19(data_path, factor)
end_time = time.time()
dask_results_dict['query_19'] = end_time - start_time

In [23]:
# query 20
start_time = time.time()
dask_queries.query_20(data_path, factor)
end_time = time.time()
dask_results_dict['query_20'] = end_time - start_time

In [24]:
# query 21
start_time = time.time()
dask_queries.query_21(data_path, factor)
end_time = time.time()
dask_results_dict['query_21'] = end_time - start_time

In [25]:
# query 22
start_time = time.time()
dask_queries.query_22(data_path, factor)
end_time = time.time()
dask_results_dict['query_22'] = end_time - start_time

In [26]:
results.append(dask_results_dict)

## Polars

In [27]:
import polars_queries

### Scale 1

In [28]:
#factor 1
factor = 1
data_path=f'./data/scale_factor_{factor}'
polars_results_dict = {'library': 'polars', 'scale': factor}

In [29]:
# query 1
start_time = time.time()
polars_queries.query_01(data_path, factor)
end_time = time.time()
polars_results_dict['query_01'] = end_time - start_time

In [30]:
# query 2
start_time = time.time()
polars_queries.query_02(data_path, factor)
end_time = time.time()
polars_results_dict['query_02'] = end_time - start_time

In [31]:
# query 3
start_time = time.time()
polars_queries.query_03(data_path, factor)
end_time = time.time()
polars_results_dict['query_03'] = end_time - start_time

In [32]:
# query 4
start_time = time.time()
polars_queries.query_04(data_path, factor)
end_time = time.time()
polars_results_dict['query_04'] = end_time - start_time

In [33]:
# query 5
start_time = time.time()
polars_queries.query_05(data_path, factor)
end_time = time.time()
polars_results_dict['query_05'] = end_time - start_time

In [34]:
# query 6
start_time = time.time()
polars_queries.query_06(data_path, factor)
end_time = time.time()
polars_results_dict['query_06'] = end_time - start_time

In [35]:
# query 7
start_time = time.time()
polars_queries.query_07(data_path, factor)
end_time = time.time()
polars_results_dict['query_07'] = end_time - start_time

In [36]:
# query 8
start_time = time.time()
polars_queries.query_08(data_path, factor)
end_time = time.time()
polars_results_dict['query_08'] = end_time - start_time

In [37]:
# query 9
start_time = time.time()
polars_queries.query_09(data_path, factor)
end_time = time.time()
polars_results_dict['query_09'] = end_time - start_time

In [38]:
# query 10
start_time = time.time()
polars_queries.query_10(data_path, factor)
end_time = time.time()
polars_results_dict['query_10'] = end_time - start_time

In [39]:
# query 11
start_time = time.time()
polars_queries.query_11(data_path, factor)
end_time = time.time()
polars_results_dict['query_11'] = end_time - start_time

In [40]:
# query 12
start_time = time.time()
polars_queries.query_12(data_path, factor)
end_time = time.time()
polars_results_dict['query_12'] = end_time - start_time

In [41]:
# query 13
start_time = time.time()
polars_queries.query_13(data_path, factor)
end_time = time.time()
polars_results_dict['query_13'] = end_time - start_time

In [42]:
# query 14
start_time = time.time()
polars_queries.query_14(data_path, factor)
end_time = time.time()
polars_results_dict['query_14'] = end_time - start_time

In [43]:
# query 15
start_time = time.time()
polars_queries.query_15(data_path, factor)
end_time = time.time()
polars_results_dict['query_15'] = end_time - start_time

In [44]:
# query 16
start_time = time.time()
polars_queries.query_16(data_path, factor)
end_time = time.time()
polars_results_dict['query_16'] = end_time - start_time

In [45]:
# query 17
start_time = time.time()
polars_queries.query_17(data_path, factor)
end_time = time.time()
polars_results_dict['query_17'] = end_time - start_time

In [46]:
# query 18
start_time = time.time()
polars_queries.query_18(data_path, factor)
end_time = time.time()
polars_results_dict['query_18'] = end_time - start_time

In [47]:
# query 19
start_time = time.time()
polars_queries.query_19(data_path, factor)
end_time = time.time()
polars_results_dict['query_19'] = end_time - start_time

In [48]:
# query 20
start_time = time.time()
polars_queries.query_20(data_path, factor)
end_time = time.time()
polars_results_dict['query_20'] = end_time - start_time

In [49]:
# query 21
start_time = time.time()
polars_queries.query_21(data_path, factor)
end_time = time.time()
polars_results_dict['query_21'] = end_time - start_time

In [50]:
# query 22
start_time = time.time()
polars_queries.query_22(data_path, factor)
end_time = time.time()
polars_results_dict['query_22'] = end_time - start_time

In [51]:
results.append(polars_results_dict)

## Pandas

In [52]:
import pandas_queries

### Scale 1

In [53]:
#factor 1
factor = 1
data_path=f'./data/scale_factor_{factor}'
pandas_results_dict = {'library': 'pandas', 'scale': factor}

In [54]:
# query 1
start_time = time.time()
pandas_queries.query_01(data_path, factor)
end_time = time.time()
pandas_results_dict['query_01'] = end_time - start_time

In [55]:
# query 2
start_time = time.time()
pandas_queries.query_02(data_path, factor)
end_time = time.time()
pandas_results_dict['query_02'] = end_time - start_time

In [56]:
# query 3
start_time = time.time()
pandas_queries.query_03(data_path, factor)
end_time = time.time()
pandas_results_dict['query_03'] = end_time - start_time

In [57]:
# query 4
start_time = time.time()
pandas_queries.query_04(data_path, factor)
end_time = time.time()
pandas_results_dict['query_04'] = end_time - start_time

In [58]:
# query 5
start_time = time.time()
pandas_queries.query_05(data_path, factor)
end_time = time.time()
pandas_results_dict['query_05'] = end_time - start_time

In [59]:
# query 6
start_time = time.time()
pandas_queries.query_06(data_path, factor)
end_time = time.time()
pandas_results_dict['query_06'] = end_time - start_time

In [60]:
# query 7
start_time = time.time()
pandas_queries.query_07(data_path, factor)
end_time = time.time()
pandas_results_dict['query_07'] = end_time - start_time

In [61]:
# query 8
start_time = time.time()
pandas_queries.query_08(data_path, factor)
end_time = time.time()
pandas_results_dict['query_08'] = end_time - start_time

In [62]:
# query 9
start_time = time.time()
pandas_queries.query_09(data_path, factor)
end_time = time.time()
pandas_results_dict['query_09'] = end_time - start_time

In [63]:
# query 10
start_time = time.time()
pandas_queries.query_10(data_path, factor)
end_time = time.time()
pandas_results_dict['query_10'] = end_time - start_time

In [64]:
# query 11
start_time = time.time()
pandas_queries.query_11(data_path, factor)
end_time = time.time()
pandas_results_dict['query_11'] = end_time - start_time

In [65]:
# query 11
start_time = time.time()
pandas_queries.query_11(data_path, factor)
end_time = time.time()
pandas_results_dict['query_11'] = end_time - start_time

In [66]:
# query 11
start_time = time.time()
pandas_queries.query_11(data_path, factor)
end_time = time.time()
pandas_results_dict['query_11'] = end_time - start_time

In [67]:
# query 12
start_time = time.time()
pandas_queries.query_12(data_path, factor)
end_time = time.time()
pandas_results_dict['query_12'] = end_time - start_time

In [68]:
# query 13
start_time = time.time()
pandas_queries.query_13(data_path, factor)
end_time = time.time()
pandas_results_dict['query_13'] = end_time - start_time

In [69]:
# query 14
start_time = time.time()
pandas_queries.query_14(data_path, factor)
end_time = time.time()
pandas_results_dict['query_14'] = end_time - start_time

In [70]:
# query 15
start_time = time.time()
pandas_queries.query_15(data_path, factor)
end_time = time.time()
pandas_results_dict['query_15'] = end_time - start_time

In [71]:
# query 16
start_time = time.time()
pandas_queries.query_16(data_path, factor)
end_time = time.time()
pandas_results_dict['query_16'] = end_time - start_time

In [72]:
# query 17
start_time = time.time()
pandas_queries.query_17(data_path, factor)
end_time = time.time()
pandas_results_dict['query_17'] = end_time - start_time

In [73]:
# query 18
start_time = time.time()
pandas_queries.query_18(data_path, factor)
end_time = time.time()
pandas_results_dict['query_18'] = end_time - start_time

In [74]:
# query 19
start_time = time.time()
pandas_queries.query_19(data_path, factor)
end_time = time.time()
pandas_results_dict['query_19'] = end_time - start_time

In [75]:
# query 20
start_time = time.time()
pandas_queries.query_20(data_path, factor)
end_time = time.time()
pandas_results_dict['query_20'] = end_time - start_time

In [76]:
# query 21
start_time = time.time()
pandas_queries.query_21(data_path, factor)
end_time = time.time()
pandas_results_dict['query_21'] = end_time - start_time

In [77]:
# query 22
start_time = time.time()
pandas_queries.query_22(data_path, factor)
end_time = time.time()
pandas_results_dict['query_22'] = end_time - start_time

In [78]:
results.append(pandas_results_dict)

## DuckDB

In [79]:
import duckdb

### Scale 1

In [80]:
scale = 1
duckdb_results_dict = {'library': 'duckdb', 'scale': factor}

In [81]:
conn = duckdb.connect()
  
conn.sql(f"CREATE TABLE customer AS FROM read_csv('./data/scale_factor_{scale}/customer.csv')")
conn.sql(f"CREATE TABLE nation AS FROM read_csv('./data/scale_factor_{scale}/nation.csv')")
conn.sql(f"CREATE TABLE lineitem AS FROM read_csv('./data/scale_factor_{scale}/lineitem.csv')")
conn.sql(f"CREATE TABLE orders AS FROM read_csv('./data/scale_factor_{scale}/orders.csv')")
conn.sql(f"CREATE TABLE supplier AS FROM read_csv('./data/scale_factor_{scale}/supplier.csv')")
conn.sql(f"CREATE TABLE region AS FROM read_csv('./data/scale_factor_{scale}/region.csv')")
conn.sql(f"CREATE TABLE part AS FROM read_csv('./data/scale_factor_{scale}/part.csv')")
conn.sql(f"CREATE TABLE partsupp AS FROM read_csv('./data/scale_factor_{scale}/partsupp.csv')")
for i in range(1,23):
    start_time = time.time()
    query = None
    with open(f'./duckdb_queries/{i}.sql') as f:
        query = f.read()
    result = conn.sql(query).df()
    end_time = time.time()
    duckdb_results_dict[f"query_{i:02d}"] = end_time - start_time

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [82]:
duckdb_results_dict

{'library': 'duckdb',
 'scale': 1,
 'query_01': 0.4549543857574463,
 'query_02': 0.06468725204467773,
 'query_03': 0.1356794834136963,
 'query_04': 0.148665189743042,
 'query_05': 0.14001154899597168,
 'query_06': 0.05953240394592285,
 'query_07': 0.10832095146179199,
 'query_08': 0.10847783088684082,
 'query_09': 0.3424222469329834,
 'query_10': 0.21378254890441895,
 'query_11': 0.02506113052368164,
 'query_12': 0.185621976852417,
 'query_13': 0.22634649276733398,
 'query_14': 0.06668329238891602,
 'query_15': 0.08187103271484375,
 'query_16': 0.05433988571166992,
 'query_17': 0.1373767852783203,
 'query_18': 0.327390193939209,
 'query_19': 0.276170015335083,
 'query_20': 0.0980367660522461,
 'query_21': 0.3835561275482178,
 'query_22': 0.07147026062011719}

In [83]:
results.append(duckdb_results_dict)

In [84]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('./data/results_1CPU.csv', index=False)